In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
pip install openpyxl

     |████████████████████████████████| 242 kB 545 kB/s 
Note: you may need to restart the kernel to use updated packages.


In [3]:
from tensorflow import keras
import cv2
from tensorflow.keras.preprocessing.image import load_img ,img_to_array
import matplotlib.pyplot as plt
from tensorflow.keras import Model
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D,AveragePooling2D,BatchNormalization
from tensorflow.keras.layers import Dense,Activation
from tensorflow.keras.layers import Flatten,Dropout,SpatialDropout2D,AveragePooling2D,GlobalAveragePooling2D
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.optimizers import Adam
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

2021-11-03 11:38:20.761045: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0


In [4]:
path = "/kaggle/input/breast-ultrasound-images-dataset/Dataset_BUSI_with_GT"
dir_list = [os.path.join(path,i) for i in os.listdir(path)]
size_dict = {}
for i,value in enumerate(dir_list):
    size_dict[os.listdir(path)[i]] = len(os.listdir(value))
size_dict 

{'benign': 891, 'normal': 266, 'malignant': 421}

In [5]:
import re
def clean(name):
    name = re.sub('[benign ().p]','',str(name))
    return name

In [6]:
df = pd.DataFrame(os.listdir(dir_list[0]))
# df = df.apply(str,axis=1)
df = df[0].apply(clean)
# print(df.info())
df = df[~df.str.contains('mask',regex =False)]
df = df.apply(int)
df_list = list(df)
type(df_list)
df_list.sort()
print(len(df_list))

437


In [7]:
img_size = 128
img_channel = 3
X_b , Xm_b , y_b = np.zeros((437,img_size,img_size,img_channel)) , np.zeros((437,img_size,img_size,img_channel)) , np.full(437,'benign') 
X_n , Xm_n , y_n = np.zeros((133,img_size,img_size,img_channel)) , np.zeros((133,img_size,img_size,img_channel)) , np.full(133,'normal') 
X_m , Xm_m , y_m = np.zeros((210,img_size,img_size,img_channel)) , np.zeros((210,img_size,img_size,img_channel)) , np.full(210,'malignant')

In [8]:
img1_path = os.path.join(os.path.join(path,'benign'),os.listdir(os.path.join(path,'benign'))[1])

pil_img = load_img(img1_path,color_mode = 'rgb',target_size=(img_size,img_size))
img = img_to_array(pil_img)
img_shape = img.shape
print(img_shape)

def img_num(filename):
    
    val = 0
    
    for i in range(len(filename)) :
        if filename[i] == '(' :
            while True :
                i += 1
                if filename[i] == ')' :
                    break
                val = (val*10) + int(filename[i])
            break
    
    return val

for tumor_path in dir_list :
    for image in os.listdir(tumor_path) :
        p = os.path.join(tumor_path, image)
        pil_img = load_img(p,color_mode = 'rgb',target_size=(img_size,img_size))         # read image as  grayscale ans resize it
        
        if image[-5] == ')' :                                   #if real image 
            
            if image[0] == 'b' :
                X_b[img_num(image)-1]+= img_to_array(pil_img)  # If image is real add it
            if image[0] == 'n' :                               # to X as benign , normal
                X_n[img_num(image)-1]+= img_to_array(pil_img)  # or malignant.
            if image[0] == 'm' :
                X_m[img_num(image)-1]+= img_to_array(pil_img)
        else :                                                 #else masked image
            
            if image[0] == 'b' :
                Xm_b[img_num(image)-1]+= img_to_array(pil_img)  # Similarly add the target
            if image[0] == 'n' :                               # mask to y.
                Xm_n[img_num(image)-1]+= img_to_array(pil_img)
            if image[0] == 'm' :
                Xm_m[img_num(image)-1]+= img_to_array(pil_img)

(128, 128, 3)


In [9]:
X = np.concatenate((X_b, X_n, X_m), axis = 0)
Xm = np.concatenate((Xm_b, Xm_n, Xm_m), axis = 0)
y = np.concatenate((y_b, y_n, y_m), axis = 0)

print(X.shape)
print(Xm.shape)
print(y.shape)
X /= 255.0
Xm /= 255.0

print(X.max())
print(Xm.min())

(780, 128, 128, 3)
(780, 128, 128, 3)
(780,)
1.0
0.0


In [10]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

encoder  = OneHotEncoder()
# y = y.toarray()
y=encoder.fit_transform(y.reshape(y.shape[0],1))

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.15,shuffle=True,random_state=42)
X_train,X_val,y_train,y_val = train_test_split(X_train,y_train,test_size=0.1,random_state=42,stratify = y_train.toarray())
Xm_train,Xm_test,ym_train,ym_test = train_test_split(Xm,y,test_size = 0.15,shuffle=True,random_state=42,stratify=y.toarray())
Xm_train,Xm_val,ym_train,ym_val = train_test_split(Xm_train,ym_train,test_size=0.1,random_state=42,stratify = ym_train.toarray())

class_list = encoder.categories_
print(X_train.shape,X_test.shape)
print(y_train.shape,y_test.shape)

(596, 128, 128, 3) (117, 128, 128, 3)
(596, 3) (117, 3)


In [11]:
base_model = tf.keras.applications.ResNet101(
    include_top=False,
    weights="imagenet",
    input_tensor=None,
    input_shape=img_shape,
    pooling=None
)

X_feat_out = base_model.output
X_feat_flatten = Flatten()(X_feat_out)

X_feat_model = Model(inputs = base_model.input,outputs = X_feat_flatten)
X_feat_train = X_feat_model.predict(X_train)
X_feat_val = X_feat_model.predict(X_val)
X_feat_test = X_feat_model.predict(X_test)

Xm_feat_train = X_feat_model.predict(Xm_train)
Xm_feat_val = X_feat_model.predict(Xm_val)
Xm_feat_test = X_feat_model.predict(Xm_test)

2021-11-03 11:38:49.029901: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-11-03 11:38:49.033632: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2021-11-03 11:38:49.084014: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-03 11:38:49.085024: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:00:04.0 name: Tesla P100-PCIE-16GB computeCapability: 6.0
coreClock: 1.3285GHz coreCount: 56 deviceMemorySize: 15.90GiB deviceMemoryBandwidth: 681.88GiB/s
2021-11-03 11:38:49.085074: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2021-11-03 11:38:49.115788: I tensorflow/stream_executor/platform/def

171450368/171446536 [==============================] - 5s 0us/step


2021-11-03 11:39:00.165536: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2021-11-03 11:39:00.179125: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 2199995000 Hz
2021-11-03 11:39:02.434993: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudnn.so.8
2021-11-03 11:39:07.999327: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.11
2021-11-03 11:39:08.864870: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublasLt.so.11


In [12]:
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.neural_network import MLPClassifier
from tensorflow.keras.layers import LSTM,Bidirectional,Reshape
from sklearn.metrics import accuracy_score,recall_score,precision_score,f1_score,roc_auc_score,cohen_kappa_score

In [13]:
df = pd.DataFrame(columns = ['classifier',"train_accuracy",'val_accuracy',"test_accuracy","f1_measure","kappa_score","recall","Precision"])

In [14]:
def eval(classifier_name,y_train,y_train_pred,y_val,y_val_pred,y_true,y_pred):
    y_train = np.argmax(y_train,axis=1)
#     y_train_pred = np.argmax(y_train_pred,axis=1)
    y_val = np.argmax(y_val,axis=1)
#     y_val_pred = np.argmax(y_val_pred,axis=1)
    y_true = np.argmax(y_true,axis=1)
#     y_pred = np.argmax(y_pred,axis=1)
    
    train_accuracy = round(accuracy_score(y_train,y_train_pred),4)
    val_accuracy = round(accuracy_score(y_val,y_val_pred),4)
    test_accuracy = round(accuracy_score(y_true,y_pred),4)
    f1_measure = round(f1_score(y_true,y_pred,average='weighted'),4)
    kappa_score = round(cohen_kappa_score(y_true,y_pred),4)
    recall = round(recall_score(y_true,y_pred,average='weighted'),4)
    precision = round(precision_score(y_true,y_pred,average='weighted'),4)
    
    score = {"classifier":classifier_name,"train_accuracy":train_accuracy , "val_accuracy":val_accuracy,"test_accuracy":test_accuracy,"f1_measure":f1_measure,"kappa_score":kappa_score,"recall":recall,"precision":precision}
#     df = df.append(score,ignore_index=True)
    df.loc[len(df.index)] = score.values()
    for e,a in score.items():
        print(e,a)
    print("--"*20)
    

In [15]:
def classifier_eval(classifier,classifier_name,X_train,y_train,X_val,y_val,X_test,y_test):
    
    classifier.fit(X_train,np.argmax(y_train,axis=1))
    
    y_train_pred = classifier.predict(X_train)
    y_val_pred = classifier.predict(X_val)
    y_test_pred = classifier.predict(X_test)
    
    eval(classifier_name,y_train,y_train_pred,y_val,y_val_pred,y_test,y_test_pred)

In [16]:
names = ['SVM',
    'Random Forest',
    'AdaBoost',
    'KNN',
    'XGBoost',
    'Bagging',
    'ANN'
        ]

classifier = [
    SVC(),
    RandomForestClassifier(),
    AdaBoostClassifier(),
    KNeighborsClassifier(),
    XGBClassifier(),
    BaggingClassifier(),
    MLPClassifier(max_iter = 400),
]

cls_list = zip(names,classifier)
clsm_list = zip(names,classifier)

In [17]:
for n,c in cls_list:
    classifier_eval(c,n,X_feat_train,y_train.toarray(),X_feat_val,y_val.toarray(),X_feat_test,y_test.toarray())

/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


classifier SVM
train_accuracy 0.5721
val_accuracy 0.5672
test_accuracy 0.4957
f1_measure 0.3286
kappa_score 0.0
recall 0.4957
precision 0.2457
----------------------------------------
classifier Random Forest
train_accuracy 1.0
val_accuracy 0.6866
test_accuracy 0.6838
f1_measure 0.6624
kappa_score 0.4454
recall 0.6838
precision 0.7286
----------------------------------------
classifier AdaBoost
train_accuracy 0.8154
val_accuracy 0.597
test_accuracy 0.6496
f1_measure 0.6373
kappa_score 0.4082
recall 0.6496
precision 0.6431
----------------------------------------
classifier KNN
train_accuracy 0.7718
val_accuracy 0.7015
test_accuracy 0.6838
f1_measure 0.6728
kappa_score 0.4693
recall 0.6838
precision 0.6734
----------------------------------------


/opt/conda/lib/python3.7/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[11:40:49] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
classifier XGBoost
train_accuracy 1.0
val_accuracy 0.6866
test_accuracy 0.7607
f1_measure 0.7549
kappa_score 0.5994
recall 0.7607
precision 0.7627
----------------------------------------
classifier Bagging
train_accuracy 0.9765
val_accuracy 0.6866
test_accuracy 0.6325
f1_measure 0.6129
kappa_score 0.3665
recall 0.6325
precision 0.6355
----------------------------------------
classifier ANN
train_accuracy 0.9295
val_accuracy 0.6866
test_accuracy 0.7521
f1_measure 0.7469
kappa_score 0.5834
recall 0.7521
precision 0.758
----------------------------------------


In [18]:
for l in base_model.layers:
    l.trainable = False

In [19]:
lstm_model = Sequential()
lstm_model.add(base_model)
lstm_model.add(Reshape((base_model.output.shape[1]*base_model.output.shape[2],base_model.output.shape[3])))
lstm_model.add(LSTM(128, dropout=0.5,recurrent_dropout=0.5))
lstm_model.add(Dense(3,activation='softmax'))

lstm_model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['acc'])

In [20]:
bidir_model = Sequential()
bidir_model.add(base_model)
bidir_model.add(Reshape((base_model.output.shape[1]*base_model.output.shape[2],base_model.output.shape[3])))
bidir_model.add(Bidirectional(LSTM(128, dropout=0.5,recurrent_dropout=0.5)))
bidir_model.add(Dense(3,activation='softmax'))

bidir_model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['acc'])

In [21]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
es = EarlyStopping(monitor='val_loss', mode='min', patience=4,restore_best_weights=True, verbose=1)

In [22]:
history = lstm_model.fit(X_train,y_train.toarray(),epochs = 20,validation_data = (X_val,y_val.toarray()),callbacks = [es])
lstm_train_predict = np.argmax(lstm_model.predict(X_train),axis=1)
lstm_val_predict = np.argmax(lstm_model.predict(X_val),axis=1)
lstm_test_predict = np.argmax(lstm_model.predict(X_test),axis=1)
eval("LSTM",y_train,lstm_train_predict,y_val,lstm_val_predict,y_test,lstm_test_predict)

Epoch 1/20
19/19 [==============================] - 16s 322ms/step - loss: 1.0916 - acc: 0.4766 - val_loss: 0.9489 - val_acc: 0.5672
Epoch 2/20
19/19 [==============================] - 2s 123ms/step - loss: 0.9580 - acc: 0.5696 - val_loss: 0.9757 - val_acc: 0.5672
Epoch 3/20
19/19 [==============================] - 2s 125ms/step - loss: 0.9408 - acc: 0.5830 - val_loss: 0.9452 - val_acc: 0.5672
Epoch 4/20
19/19 [==============================] - 2s 123ms/step - loss: 0.9559 - acc: 0.5649 - val_loss: 0.8997 - val_acc: 0.6119
Epoch 5/20
19/19 [==============================] - 3s 143ms/step - loss: 0.9000 - acc: 0.6051 - val_loss: 0.8721 - val_acc: 0.6269
Epoch 6/20
19/19 [==============================] - 2s 122ms/step - loss: 0.9074 - acc: 0.5829 - val_loss: 0.9747 - val_acc: 0.6119
Epoch 7/20
19/19 [==============================] - 2s 129ms/step - loss: 0.9122 - acc: 0.6016 - val_loss: 0.8610 - val_acc: 0.6567
Epoch 8/20
19/19 [==============================] - 2s 125ms/step - loss: 0

/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [23]:
history = bidir_model.fit(X_train,y_train.toarray(),epochs = 20,validation_data = (X_val,y_val.toarray()),callbacks = [es])
bidir_train_predict = np.argmax(bidir_model.predict(X_train),axis=1)
bidir_val_predict = np.argmax(bidir_model.predict(X_val),axis=1)
bidir_test_predict = np.argmax(bidir_model.predict(X_test),axis=1)
eval("Bi-dir",y_train,bidir_train_predict,y_val,bidir_val_predict,y_test,bidir_test_predict)

Epoch 1/20
19/19 [==============================] - 20s 390ms/step - loss: 1.0110 - acc: 0.5274 - val_loss: 0.9509 - val_acc: 0.5672
Epoch 2/20
19/19 [==============================] - 4s 209ms/step - loss: 0.9635 - acc: 0.5694 - val_loss: 0.8838 - val_acc: 0.6269
Epoch 3/20
19/19 [==============================] - 5s 267ms/step - loss: 0.9703 - acc: 0.5668 - val_loss: 0.9042 - val_acc: 0.6269
Epoch 4/20
19/19 [==============================] - 4s 227ms/step - loss: 0.9000 - acc: 0.6088 - val_loss: 0.8745 - val_acc: 0.6418
Epoch 5/20
19/19 [==============================] - 4s 231ms/step - loss: 0.9134 - acc: 0.5956 - val_loss: 0.8581 - val_acc: 0.6418
Epoch 6/20
19/19 [==============================] - 4s 210ms/step - loss: 0.8637 - acc: 0.6199 - val_loss: 0.8603 - val_acc: 0.6567
Epoch 7/20
19/19 [==============================] - 4s 212ms/step - loss: 0.8623 - acc: 0.6470 - val_loss: 0.8691 - val_acc: 0.6567
Epoch 8/20
19/19 [==============================] - 4s 204ms/step - loss: 0

In [24]:
df

,classifier,train_accuracy,val_accuracy,test_accuracy,f1_measure,kappa_score,recall,Precision
0,SVM,0.5721,0.5672,0.4957,0.3286,0.0000,0.4957,0.2457
1,Random Forest,1.0000,0.6866,0.6838,0.6624,0.4454,0.6838,0.7286
2,AdaBoost,0.8154,0.5970,0.6496,0.6373,0.4082,0.6496,0.6431
3,KNN,0.7718,0.7015,0.6838,0.6728,0.4693,0.6838,0.6734
4,XGBoost,1.0000,0.6866,0.7607,0.7549,0.5994,0.7607,0.7627
5,Bagging,0.9765,0.6866,0.6325,0.6129,0.3665,0.6325,0.6355
6,ANN,0.9295,0.6866,0.7521,0.7469,0.5834,0.7521,0.7580
7,LSTM,0.6258,0.6567,0.5897,0.5101,0.2418,0.5897,0.4855
8,Bi-dir,0.6275,0.6418,0.5812,0.5180,0.2616,0.5812,0.4685


In [25]:
df.to_excel("RESNET101.xlsx",index=False)

In [26]:
df = pd.DataFrame(columns = ['classifier',"train_accuracy",'val_accuracy',"test_accuracy","f1_measure","kappa_score","recall","Precision"])
for n,c in clsm_list:
    classifier_eval(c,n,Xm_feat_train,ym_train.toarray(),Xm_feat_val,ym_val.toarray(),Xm_feat_test,ym_test.toarray())

classifier SVM
train_accuracy 0.8641
val_accuracy 0.806
test_accuracy 0.8632
f1_measure 0.8561
kappa_score 0.7573
recall 0.8632
precision 0.8677
----------------------------------------
classifier Random Forest
train_accuracy 1.0
val_accuracy 0.8209
test_accuracy 0.906
f1_measure 0.9056
kappa_score 0.8378
recall 0.906
precision 0.9053
----------------------------------------
classifier AdaBoost
train_accuracy 0.8372
val_accuracy 0.7313
test_accuracy 0.8205
f1_measure 0.8259
kappa_score 0.706
recall 0.8205
precision 0.8478
----------------------------------------
classifier KNN
train_accuracy 0.9178
val_accuracy 0.806
test_accuracy 0.8547
f1_measure 0.8526
kappa_score 0.7472
recall 0.8547
precision 0.852
----------------------------------------


/opt/conda/lib/python3.7/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[11:53:03] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
classifier XGBoost
train_accuracy 1.0
val_accuracy 0.8806
test_accuracy 0.8889
f1_measure 0.8891
kappa_score 0.8111
recall 0.8889
precision 0.8898
----------------------------------------
classifier Bagging
train_accuracy 0.9916
val_accuracy 0.8507
test_accuracy 0.9145
f1_measure 0.9128
kappa_score 0.8506
recall 0.9145
precision 0.9142
----------------------------------------
classifier ANN
train_accuracy 1.0
val_accuracy 0.8806
test_accuracy 0.9231
f1_measure 0.9227
kappa_score 0.8673
recall 0.9231
precision 0.9226
----------------------------------------


In [27]:
history = lstm_model.fit(Xm_train,ym_train.toarray(),epochs = 20,validation_data = (Xm_val,ym_val.toarray()),callbacks = [es])
lstm_train_predict = np.argmax(lstm_model.predict(Xm_train),axis=1)
lstm_val_predict = np.argmax(lstm_model.predict(Xm_val),axis=1)
lstm_test_predict = np.argmax(lstm_model.predict(Xm_test),axis=1)
eval("LSTM",ym_train,lstm_train_predict,ym_val,lstm_val_predict,ym_test,lstm_test_predict)

history = bidir_model.fit(Xm_train,ym_train.toarray(),epochs = 20,validation_data = (Xm_val,ym_val.toarray()),callbacks = [es])
bidir_train_predict = np.argmax(bidir_model.predict(Xm_train),axis=1)
bidir_val_predict = np.argmax(bidir_model.predict(Xm_val),axis=1)
bidir_test_predict = np.argmax(bidir_model.predict(Xm_test),axis=1)
eval("Bi-LSTM",ym_train,bidir_train_predict,ym_val,bidir_val_predict,ym_test,bidir_test_predict)

Epoch 1/20
19/19 [==============================] - 2s 128ms/step - loss: 1.1072 - acc: 0.5067 - val_loss: 0.8763 - val_acc: 0.5970
Epoch 2/20
19/19 [==============================] - 3s 149ms/step - loss: 0.7889 - acc: 0.6107 - val_loss: 0.7031 - val_acc: 0.7164
Epoch 3/20
19/19 [==============================] - 2s 121ms/step - loss: 0.5695 - acc: 0.7534 - val_loss: 0.5605 - val_acc: 0.7761
Epoch 4/20
19/19 [==============================] - 3s 166ms/step - loss: 0.4556 - acc: 0.8054 - val_loss: 0.6686 - val_acc: 0.7612
Epoch 5/20
19/19 [==============================] - 2s 121ms/step - loss: 0.3692 - acc: 0.8557 - val_loss: 0.4383 - val_acc: 0.8060
Epoch 6/20
19/19 [==============================] - 3s 144ms/step - loss: 0.3104 - acc: 0.8775 - val_loss: 0.4167 - val_acc: 0.8657
Epoch 7/20
19/19 [==============================] - 2s 122ms/step - loss: 0.3220 - acc: 0.8725 - val_loss: 0.4003 - val_acc: 0.8060
Epoch 8/20
19/19 [==============================] - 2s 122ms/step - loss: 0.

In [28]:
print(df)
df.to_excel("RESNET101_m.xlsx",index=False)

      classifier  train_accuracy  val_accuracy  test_accuracy  f1_measure  \
0            SVM          0.8641        0.8060         0.8632      0.8561   
1  Random Forest          1.0000        0.8209         0.9060      0.9056   
2       AdaBoost          0.8372        0.7313         0.8205      0.8259   
3            KNN          0.9178        0.8060         0.8547      0.8526   
4        XGBoost          1.0000        0.8806         0.8889      0.8891   
5        Bagging          0.9916        0.8507         0.9145      0.9128   
6            ANN          1.0000        0.8806         0.9231      0.9227   
7           LSTM          0.9346        0.8060         0.9060      0.9071   
8        Bi-LSTM          0.9211        0.8358         0.8803      0.8792   

   kappa_score  recall  Precision  
0       0.7573  0.8632     0.8677  
1       0.8378  0.9060     0.9053  
2       0.7060  0.8205     0.8478  
3       0.7472  0.8547     0.8520  
4       0.8111  0.8889     0.8898  
5       0.850